# Exercises - week 2

Write your solutions to the given cells (see "WRITE YOUR CODE HERE"). Make your code print only what is requested. Select `Kernel` &rightarrow; `Restart & Clear Output` and then `Cell` &rightarrow; `Run All` from the menu before submitting your solutions, so that your submission contains clean output produced from scratch.

#### Exercise 2.1

Print the ids of those entries that match the protein name "p53" and the organism "Homo sapiens" in the NCBI Protein database, one per line.

In [2]:
import Bio.Entrez as BE
import Bio.SeqIO as BSIO
# set email globally
BE.email = 'jojuket@utu.fi'
# remember to set your real email address
# before continuing to the examples below
#SET_YOUR_REAL_EMAIL_ADDRESS_BEFORE_COMMENTING_OUT_THIS_LINE

In [2]:
handle = BE.esearch(db="protein", term="Homo sapiens[Organism] p53[Gene Name]")
# get response from NCBI
# (data as bytes, not as string)
results = handle.read()
# close the handle
handle.close()
# save results to file
# ('wb' needed to write bytes)
with open('results.xml', 'wb') as f:
    f.write(results)
    # load from file and parse
# ('rb' needed to read data as bytes)
with open('results.xml', 'rb') as f:
    results = BE.read(f)
# report the retrieved IDs
# print(results['IdList'])
    
    # Probably should've done this somehow else but didn't know how so my style was:
    # Extract the IdList from the results
    pIDS = results['IdList']
    print("ID's are:")
    #make a for loop to print the ID's one by one
    for i in pIDS:
        print(i)

 

ID's are:
454545707
454545207
454544715
454544286
454543597
454521557
454520873
371502119
187830909
187830901
187830894
187830855
187830823
187830777
120407068
269849759
232816
232814
189476
349734072


#### Exercise 2.2

Fetch the entry NP_004955 from the NCBI Protein database and save it to a file named `NP_004955.gb`. Use the GenPept flat file format.

In [3]:
# fetch records from "protein" database
# and return sequences in GenBank flat file format
handle = BE.efetch(db="protein",
                   id="NP_004955",
                   rettype="gb",
                   retmode="text")
# get response from NCBI
# (data as string, not bytes)
results = handle.read()
# close the handle
handle.close()
# save to file
# ('w' needed to write string)
with open('NP_004955.gb', 'w') as f:
    f.write(results)
# show raw file content to check that the download was a succé 
with open('NP_004955.gb') as f:
    print(f.read())

LOCUS       NP_004955                482 aa            linear   PRI 18-JAN-2021
DEFINITION  histone deacetylase 1 [Homo sapiens].
ACCESSION   NP_004955
VERSION     NP_004955.2
DBSOURCE    REFSEQ: accession NM_004964.3
KEYWORDS    RefSeq; MANE Select.
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (residues 1 to 482)
  AUTHORS   Lv L, Zhang D, Hua P and Yang S.
  TITLE     The glial-specific hypermethylated 3' untranslated region of
            histone deacetylase 1 may modulates several signal pathways in
            Alzheimer's disease
  JOURNAL   Life Sci 265, 118760 (2021)
   PUBMED   33212149
  REMARK    GeneRIF: The glial-specific hypermethylated 3' untranslated region
            of histone deacetylase 1 may modulates several signal pathways in
            Alzheimer

#### Exercise 2.3

Load the file `NP_004955.gb` and print the following pieces of information about the protein, one per line.
- id
- name
- description
- organism
- number of features

In [4]:
# parse entries in GenBank flat file format
p53Rec = BSIO.parse('NP_004955.gb', 'gb')
#using string formatting %s and %i combined to a for loop to get the required info

for f in p53Rec:
    print(" ID: %s\n Name: %s\n Description: %s\n Organism: %s\n Number of features: %i" 
          % (f.id, f.name, f.description, f.annotations['organism'], len(f.features)))  

 ID: NP_004955.2
 Name: NP_004955
 Description: histone deacetylase 1 [Homo sapiens]
 Organism: Homo sapiens
 Number of features: 15


#### Exercise 2.4

Print the number of phosphorylation sites annotated in the NCBI Protein database entry NP_004955.

In [19]:
from collections import Counter
i = []
for rec in BSIO.parse('NP_004955.gb', 'gb'):
    #make rec to loop over the NP_004955.gb file
    for feature in rec.features:
        #move into features
        if feature.type == "Site":
            # get only values from keyword "Site"
            i= i+feature.qualifiers["site_type"]
            #append the values from the "site_type" to a list i
#initalize the counter with the list i            
c = Counter(i)
#count the occurences of word phosphorylation
print("Phosphorylated sites: "+ str(c['phosphorylation']))
            
# This was a bit of a turn-a-round, but basically it seems I could do it like this

Phosphorylated sites: 5


#### Exercise 2.5

Regarding the NCBI Protein database entry NP_004955, print the phosphorylated amino acid residues and their positions in the sequence, one per line.

In [13]:
#Actually couldn't be more proud of myself, I did it, I solved the problem!!!!!!!!
#Maybe too early to say, but I feel that i actually learnt something!!!

# Make an SeqFeature? object
for rec in BSIO.parse('NP_004955.gb', 'gb'):
    # intialize variable feature to go through the features of the Seq object
    for feature in rec.features:
        #Determine that the we are looking for the word "Site"
        if feature.type == "Site":
            # initialize variable x to go through site_types and get the values
            for x in feature.qualifiers.get("site_type"):
                # the key word is "phosphorylation"
                if x == "phosphorylation":
                    #then print the site_type x and the location of it
                    print(x + str(feature.location))
                    #print(feature.qualifiers.get("note"))
                    
#If i try to print the name of the phosphorylated residue as in print(feature.qualifiers.get("note")) 
#The Jupyter prints out both /note and /evidence. Could this be because the NCBI data doesn't have "" around 
#the name of the amino acid residue

phosphorylation[392:393]
phosphorylation[405:406]
phosphorylation[408:409]
phosphorylation[420:421]
phosphorylation[422:423]


In [ ]:
def get_feature(features, id, tags=["locus_tag", "gene"]):
    """Search list of SeqFeature objects for an identifier under the given tags."""
    for f in features:
        for key in tags:
            # tag may not be present in this feature
            for x in f.qualifiers.get(key, []):
                if x == id:
                    return f
    raise KeyError(id)

#### Exercise 2.6

The NCBI Nucleotide database entry 223468685 contains several genes. Print their names in the order of appearance, one per line.

In [3]:
# fetch records from "nucleotide" database
# and return sequences in GenBank flat file format
handle = BE.efetch(db="nucleotide",
                   id="223468685",
                   rettype="gb",
                   retmode="text")
# get response from NCBI
# (data as string, not bytes)
results = handle.read()
# close the handle
handle.close()
# save to file
# ('w' needed to write string)
with open('geneEntry.gb', 'w') as f:
    f.write(results)
    
#as in 2.5, but this time get the gene entries
for rec in BSIO.parse('geneEntry.gb', 'gb'):
    for feature in rec.features:
        if feature.type == "gene":
            #0 as there is only one entry in the list
            print (feature.qualifiers["gene"][0])
            #print (feature.location)

RPL36A
RPL36A-HNRNPH2
BTK
TIMM8A


In [40]:
#just to see the keywords for BTK
with open('geneEntry.gb') as f:
    print(f.read())

LOCUS       NG_009616              43778 bp    DNA     linear   PRI 22-AUG-2020
DEFINITION  Homo sapiens Bruton tyrosine kinase (BTK), RefSeqGene (LRG_128) on
            chromosome X; nuclear gene for mitochondrial product.
ACCESSION   NG_009616
VERSION     NG_009616.1
KEYWORDS    RefSeq; RefSeqGene.
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 43778)
  AUTHORS   Smith,C.I.E. and Berglof,A.
  TITLE     X-Linked Agammaglobulinemia
  JOURNAL   (in) Adam MP, Ardinger HH, Pagon RA, Wallace SE, Bean LJH, Stephens
            K and Amemiya A (Eds.);
            GENEREVIEWS((R));
            (1993)
   PUBMED   20301626
COMMENT     REVIEWED REFSEQ: This record has been curated by NCBI staff in
            collaboration with Jouni Valiaho, Mauno Vihinen. The referen

#### Exercise 2.7

Print the peptide sequence produced by the gene named "BTK" in the NCBI Nucleotide database entry 223468685.

(Note that the CDS is discontinuous and consists of several regions, which is modelled by a `CompoundLocation` object.)

In [4]:
# Make an SeqFeature? object
for rec in BSIO.parse('geneEntry.gb', 'gb'):
    # intialize variable feature to go through the features of the Seq object
    for feature in rec.features:
        #Determine that the we are looking for the word "CDS"
        if feature.type == "CDS":
            # initialize variable x to go through gene names under CDS
            for x in feature.qualifiers.get("gene"):
                # the word is "BTK"
                if x == "BTK":
                    #then print the site_type x and the location of it
                    print(x)
                    print(feature.qualifiers["translation"][0])
                    #just to try if I can convert this list to string
                    print("This is a string: \n" + ' '.join([str(elem) for elem in feature.qualifiers["translation"]]) )






#for rec in BSIO.parse('geneEntry.gb', 'gb'):
 #   for feature in rec.features:
  #      if feature.type == "CDS":
   #         print (feature.qualifiers["translation"])
            #print (feature.location)

BTK
MAAVILESIFLKRSQQKKKTSPLNFKKRLFLLTVHKLSYYEYDFERGRRGSKKGSIDVEKITCVETVVPEKNPPPERQIPRRGEESSEMEQISIIERFPYPFQVVYDEGPLYVFSPTEELRKRWIHQLKNVIRYNSDLVQKYHPCFWIDGQYLCCSQTAKNAMGCQILENRNGSLKPGSSHRKTKKPLPPTPEEDQILKKPLPPEPAAAPVSTSELKKVVALYDYMPMNANDLQLRKGDEYFILEESNLPWWRARDKNGQEGYIPSNYVTEAEDSIEMYEWYSKHMTRSQAEQLLKQEGKEGGFIVRDSSKAGKYTVSVFAKSTGDPQGVIRHYVVCSTPQSQYYLAEKHLFSTIPELINYHQHNSAGLISRLKYPVSQQNKNAPSTAGLGYGSWEIDPKDLTFLKELGTGQFGVVKYGKWRGQYDVAIKMIKEGSMSEDEFIEEAKVMMNLSHEKLVQLYGVCTKQRPIFIITEYMANGCLLNYLREMRHRFQTQQLLEMCKDVCEAMEYLESKQFLHRDLAARNCLVNDQGVVKVSDFGLSRYVLDDEYTSSVGSKFPVRWSPPEVLMYSKFSSKSDIWAFGVLMWEIYSLGKMPYERFTNSETAEHIAQGLRLYRPHLASEKVYTIMYSCWHEKADERPTFKILLSNILDVMDEES
This is a string: 
MAAVILESIFLKRSQQKKKTSPLNFKKRLFLLTVHKLSYYEYDFERGRRGSKKGSIDVEKITCVETVVPEKNPPPERQIPRRGEESSEMEQISIIERFPYPFQVVYDEGPLYVFSPTEELRKRWIHQLKNVIRYNSDLVQKYHPCFWIDGQYLCCSQTAKNAMGCQILENRNGSLKPGSSHRKTKKPLPPTPEEDQILKKPLPPEPAAAPVSTSELKKVVALYDYMPMNANDLQLRKGDEYFILEESNLPWWRARDKNGQEGYIPSNYVTEAEDSIEMYEWYSKHMTRSQAEQLLKQEGKEGGFIVRDSSKAGKYTV

#### Exercise 2.8

Fetch the sequences of those NCBI Protein database entries that concern the species Saccharomyces boulardii and save them to a file named `saccharomyces-boulardii-proteins.fasta` in the FASTA format. Use the query history and fetch the sequences in batches of 100 sequences.

In [18]:
#need to determine the count as len(0, count,100)
#that way it increases always by 100
def retrieve(query, batch_size):
    # search using history
    handle = BE.esearch(db="protein",
                        term=query,
                        usehistory="y")
    results = BE.read(handle)
    handle.close()
    webenv = results['WebEnv']
    querykey = results['QueryKey']
 # adjust maximum size if less than maximum number of hits available
   # retrieve_size = min(int(results['Count']), retrieve_size)
    for i in range(0, batch_size):
        handle = BE.efetch(db="protein",
            webenv=webenv,
            query_key=querykey,
                           # fetch N records starting from i
            retstart=i,
            retmax=batch_size,
            rettype="fasta",
            retmode="text")
        for record in BSIO.parse(handle, 'fasta'):
            yield record
        handle.close()
        
for record in retrieve("Saccharomyces boulardii", 100):
    BSIO.write([record], 'saccharomyces-boulardii-proteins.fasta', 'fasta')
   # print(record)
 
#Didn't test if the file is readable as there are so many entries and it was said in the
#wednesday session taht there wasn't a need for printing


#with open('saccharomyces-boulardii-proteins.fasta') as f:
  #  print(f.read())

KeyboardInterrupt: 